## 0. Import lib

In [1]:
import cv2
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt

## 1. Define variable

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


## 2. Detect Left & Right hands

In [7]:
def getLabel(result, hands_lms, hand_flag):
    output = None
    for idx, classification in enumerate(result.multi_handedness):
        if classification.classification[0].index == hand_flag:
            label = classification.classification[0].label
            coords = tuple(np.multiply(
                np.array((hands_lms.landmark[mp_hands.HandLandmark.WRIST].x, hands_lms.landmark[mp_hands.HandLandmark.WRIST].y)),
                [640, 480]).astype(int))
            output = label, coords
    return output
        

## 3. Render hands

In [18]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        # read image
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        # process image
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # detection
        if result.multi_hand_landmarks:
            for id, hands_lms in enumerate(result.multi_hand_landmarks):
                label = result.multi_handedness[id].classification[0].label
                coord = tuple(np.multiply(
                np.array((hands_lms.landmark[mp_hands.HandLandmark.WRIST].x, hands_lms.landmark[mp_hands.HandLandmark.WRIST].y)),
                [640, 480]).astype(int))
                mp_drawing.draw_landmarks(img, hands_lms, mp_hands.HAND_CONNECTIONS)
        # classification
                cv2.putText(img, label, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Hello', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))